In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar  5 16:01:19 2019

@author: Arsene Gasana
"""
#import modules
import pandas as pd 
import xlsxwriter as xwt
import numpy as np
import os
from datetime import datetime
version = '1_0'

main_dir = r'D:\IDS PANDAS OUTPUT'
os.chdir(main_dir)
time = datetime.now()
datestring_for_file = '%04d%02d%02d%02d%02d%02d' % (time.year, time.month, time.day, time.hour,time.minute, time.second)

#import excel database and organize it
df = pd.read_csv(r'C:\Users\user\Desktop\chicken IDS tes.csv')

df['SiteID'] = df['DistrictName']+df['SiteName']
df['GroupID'] = df['SiteName'] + df['GroupName']

#get a list of districts and make directories per district & get a list of sites
GetDistrictList = np.unique(df['DistrictName'].values)
for dist in GetDistrictList:
    Dist_dir_name = '%s-%s-%s' % (dist,version, datestring_for_file)
    os.chdir(main_dir)
    os.mkdir(Dist_dir_name)
    #os.chdir(main_dir +'\\'+ Dist_dir_name)
    DistrictDF = df.loc[df['DistrictName']==dist]
    site_list=np.unique(DistrictDF['SiteName'].values)

    for site in site_list:
        Sitedirname = '%s-%s-%s' % (site,version, datestring_for_file)
        os.chdir(main_dir +'\\'+ Dist_dir_name)
        os.mkdir(Sitedirname)
        os.chdir(main_dir +'\\'+ Dist_dir_name+'\\'+Sitedirname)
        SiteDF = df.loc[df['SiteName']==site]
        Group_list=np.unique(SiteDF['GroupName'].values)
        
        for group in Group_list :
            mydf = df.loc[df['GroupName']==group]
            Akarere = mydf['DistrictName'].tolist()[0]
            Akagali = mydf['SiteName'].tolist()[0]
            Itsinda = mydf['GroupName'].tolist()[0]
            finaldf = mydf[['OAFID','LastName','FirstName','2021A_Inkoko qty']]
            
            finaldf['indexName'] = finaldf.index.tolist()
            finaldf['indexName'].astype(str)
            indexDf = pd.DataFrame({'index_list':finaldf.index.tolist(), 'index_name':'  '})
            indexDf['indexName']=indexDf['index_list'].astype(str) + indexDf['index_name']

            GroupOutput = finaldf.append(indexDf, ignore_index=False)
            GroupOutput['indexName'] = GroupOutput['indexName'].astype(str)
            GroupOutput = GroupOutput.sort_values(by='indexName')
            GroupOutput['FirstName']=GroupOutput['FirstName'].fillna('Ibihindurwa')
            GroupOutput = GroupOutput.fillna(' ')
            GroupOutput = GroupOutput[['OAFID','LastName','FirstName','2021A_Inkoko qty']]
            
            
            GroupPDF = [GroupOutput.columns.values.tolist()] + GroupOutput.values.tolist()
            fileName = group+'.pdf'

            from reportlab.platypus import SimpleDocTemplate
            from reportlab.lib.pagesizes import letter
            from reportlab.platypus import  Table
            from reportlab.platypus import TableStyle,Frame
            from reportlab.lib import colors
            from reportlab.pdfgen import canvas

            #pdf = SimpleDocTemplate(fileName, pagesize=letter,rightMargin=72,leftMargin=72,topMargin=72,bottomMargin=18)
            pdf = canvas.Canvas(fileName)
            table = Table(GroupPDF)

            pdf.setTitle('Hello')

            rowNumb = len(GroupPDF)
            for i in range(1, rowNumb):
                if i%2 == 0:
                    bc = colors.beige
                    style = TableStyle([('BACKGROUND',(0,i),(3,i),bc),
                                        ('SPAN',(0,i),(-3,i)),
                                        ('FONTNAME',(0,i), (-1,i),'Times-Italic'),
                                        ('FONTSIZE', (0,i), (-1,i), 9)])
                    table.setStyle(style)
                else:
                    bc = colors.white
                    ts = TableStyle([('GRID',(0,0),(-1,-1),1,colors.black),
                                     ('FONTNAME',(0,0), (-1,-1),'Times-Roman'),
                                     ('FONTSIZE', (0,0), (-1,-1), 9)])

                    table.setStyle(ts) 

            ExportList = []
            ExportList.append(table)

            frame = Frame(30,20,300, 750, showBoundary=0)
            frame.addFromList(ExportList,pdf)

            pdf.drawString(45, 820, 'Akarere:  '+Akarere)
            pdf.drawString(45, 800, 'Akagali:  '+Akagali)
            pdf.drawString(45, 780, 'Itsinda:  '+Itsinda)
            pdf.drawImage(r'C:\Users\user\Desktop\TaburaRwanda.jpg', 450 ,790, 100, 50)


            pdf.save()


In [ ]:
"""

from reportlab.platypus import SimpleDocTemplate
from reportlab.lib.pagesizes import letter
from reportlab.platypus import  Table
from reportlab.platypus import TableStyle,Frame
from reportlab.lib import colors
from reportlab.pdfgen import canvas

pdf = SimpleDocTemplate(fileName, pagesize=letter,rightMargin=72,leftMargin=72,topMargin=72,bottomMargin=18)
#pdf = canvas.Canvas(fileName)
table = Table(GroupPDF)

rowNumb = len(GroupPDF)
for i in range(1, rowNumb):
    if i%2 == 0:
        bc = colors.beige
        style = TableStyle([('BACKGROUND',(0,i),(3,i),bc),
                            ('SPAN',(0,i),(-3,i)),
                            ('FONTNAME',(0,i), (-1,i),'Times-Italic'),
                            ('FONTSIZE', (0,i), (-1,i), 9)])
        table.setStyle(style)
    else:
        bc = colors.white
        ts = TableStyle([('GRID',(0,0),(-1,-1),1,colors.black),
                         ('FONTNAME',(0,0), (-1,-1),'Times-Roman'),
                         ('FONTSIZE', (0,0), (-1,-1), 9)])
        
        table.setStyle(ts) 

ExportList = []
ExportList.append(table)

#frame = Frame(30,20,300, 750, showBoundary=0)
#frame.addFromList(ExportList,pdf)

pdf.drawString(45, 820, 'Akarere:  '+Akarere)
pdf.drawString(45, 800, 'Akagali:  '+Akagali)
pdf.drawString(45, 780, 'Itsinda:  '+Itsinda)
pdf.drawImage(r'C:\Users\user\Desktop\TaburaRwanda.jpg', 450 ,790, 100, 50)

pdf.build(ExportList)
#pdf.save()
"""